In [183]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [184]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [185]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [186]:
%%capture pwd
!pwd

In [187]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-are-the-economic-effects-of-extreme-weather-caused-by-climate-change',
 300,
 500)

# Fig 1

## a

In [188]:
# https://www.metoffice.gov.uk/weather/climate/climate-and-extreme-weather
# https://www.munichre.com/en/risks/natural-disasters-losses-are-trending-upwards.html?#1995343501
# https://www.munichre.com//en/risks/natural-disasters-losses-are-trending-upwards/_jcr_content/root/content/chart.chartdata.json
data = json.loads(open("raw/data.json", "r").read())
df = pd.DataFrame([i["data"] for i in data]).T
df.columns = [i["label"] for i in data]
df.index = range(1980, 2020)
df = df.reset_index()
df.columns = ["year", "total", "hydro", "meteo", "climate", "geo"]
df = df.set_index("year").stack().reset_index()
df.columns = ["year", "data", "value"]

In [189]:
f = "fig1a_extreme-events"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,data,value
0,1980,total,249.0
1,1980,hydro,59.0
2,1980,meteo,135.0
3,1980,climate,28.0
4,1980,geo,27.0


In [ ]:
base = alt.Chart(f1a).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
bars = (
    base.mark_bar()
    .encode(
        y=alt.Y(
            "value:Q",
            stack=True,
            axis=alt.Axis(
                grid=True,
                title="number of events",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".0f",
            ),
        ),
        color=alt.Color(
            "data:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-mid-blue"],
                    colors["eco-turquiose"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter('datum.data!="total"')
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 2020, "y": 700, "t": "Geophysical events:", "d": "geo"},
                {"x": 2020, "y": 660, "t": "earthquakes, tsunami", "d": "geo"},
                {"x": 2020, "y": 620, "t": "volcanic activity", "d": "geo"},
                {"x": 2020, "y": 850, "t": "Climatological events:", "d": "climate"},
                {"x": 2020, "y": 810, "t": "extreme temperature,", "d": "climate"},
                {"x": 2020, "y": 770, "t": "drought, wildfire", "d": "climate"},
                {"x": 2020, "y": 240, "t": "Meteorological events:", "d": "meteo"},
                {"x": 2020, "y": 200, "t": "tropical storm,", "d": "meteo"},
                {"x": 2020, "y": 160, "t": "extratropical storm,", "d": "meteo"},
                {"x": 2020, "y": 120, "t": "convective storm,", "d": "meteo"},
                {"x": 2020, "y": 80, "t": "local storm", "d": "meteo"},
                {"x": 2020, "y": 500, "t": "Hydrological events:", "d": "hydro"},
                {"x": 2020, "y": 460, "t": "flood, mass movement", "d": "hydro"},
            ]
        )
    )
    .mark_text(align="left")
    .encode(x="x:Q", y="y:Q", text="t:N", color="d:N")
)
title = alt.TitleParams(
    "Extreme events are becoming more frequent",
    subtitle=[
        "Number of relevant loss events by peril. Source: Met Office, based on Munich RE"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((bars + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

## b

In [ ]:
f = "fig1a_extreme-event-dotplot"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f1b).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
        scale=alt.Scale(domain=[1980, 2020]),
    )
)
dots = (
    base.mark_point()
    .encode(
        y=alt.Y("data:N", stack=True, axis=None),
        size=alt.Size("value:Q", legend=None, scale=alt.Scale(domain=[15, 150])),
        fill=alt.Fill(
            "data:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-mid-blue"],
                    colors["eco-turquiose"],
                    colors["eco-gray"],
                ]
            ),
        ),
        color=alt.Color(
            "data:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-light-blue"],
                    colors["eco-mid-blue"],
                    colors["eco-turquiose"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter('datum.data!="total"')
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 1980, "y": 700, "t": "Geophysical events:", "d": "geo"},
                {"x": 1980, "y": 850, "t": "Climatological events:", "d": "climate"},
                {"x": 1980, "y": 240, "t": "Meteorological events:", "d": "meteo"},
                {"x": 1980, "y": 500, "t": "Hydrological events:", "d": "hydro"},
            ]
        )
    )
    .mark_text(align="left", fontWeight='bold',dy=-16, dx=-6)
    .encode(x="x:Q", y="d:N", text="t:N", color="d:N")
)
rlabels = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": 1985.6,
                    "y": 660,
                    "t": "earthquakes, tsunami, volcanic activity",
                    "d": "geo",
                },
                {
                    "x": 1986.1,
                    "y": 810,
                    "t": "extreme temperature, drought, wildfire",
                    "d": "climate",
                },
                {
                    "x": 1986.2,
                    "y": 200,
                    "t": "tropical storm, extratropical storm, convective storm, local storm",
                    "d": "meteo",
                },
                {"x": 1985.6, "y": 460, "t": "flood, mass movement", "d": "hydro"},
            ]
        )
    )
    .mark_text(align="left", dy=-16, dx=8)
    .encode(x="x:Q", y="d:N", text="t:N", color="d:N")
)
title = alt.TitleParams(
    "Extreme events are becoming more frequent",
    subtitle=[
        "Circle size is proportional to the number of relevant loss events by peril type and year. Source: Met Office, based on Munich RE"
    ],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    dy=-10,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((dots + labels + rlabels).properties(height=150, width=620))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1